In [2]:
import csv
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from sklearn.metrics import log_loss
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier 


In [3]:
df_match_allyears = pd.read_csv("./match_data_yearly/all_years.csv", index_col=0)
df_match_allyears["Date"] = pd.to_datetime(df_match_allyears["Date"])

In [4]:
df_match_allyears

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,Stadium,Attendances
ID,,,,,,,,,,,,
6010903,2006,1,2006-03-04,gamba-osaka,urawa-red-diamonds,9,3,1,1,0,万博,20916
6012807,2006,1,2006-03-05,ventforet-kofu,shimizu-s-pulse,28,7,0,2,2,小瀬,14277
6012231,2006,1,2006-03-05,fc-tokyo,oita-trinita,22,31,2,0,1,味スタ,22531
6011323,2006,1,2006-03-05,jubilo-iwata,avispa-fukuoka,13,23,1,1,0,静岡,28564
6010820,2006,1,2006-03-05,nagoya-grampus-eight,cerezo-osaka,8,20,3,2,1,瑞穂陸,17199
...,...,...,...,...,...,...,...,...,...,...,...,...
20340120,2020,34,2020-12-19,kashima-antlers,cerezo-osaka,1,20,1,1,0,カシマ,11251
20340810,2020,34,2020-12-19,nagoya-grampus-eight,sanfrecce-hiroshima,8,10,1,0,1,豊田ス,16825
20343331,2020,34,2020-12-19,sagan-tosu,oita-trinita,33,31,2,2,0,駅スタ,8966


In [5]:
df_recent_stats = pd.read_csv("./recent_stats.csv", index_col=0)
df_recent_stats

,HomeShots,HomeShotsOnTarget,HomeShotsFromPK,HomePasses,HomeCrosses,HomeDirectFK,HomeIndirectFK,HomeCK,HomeThrowin,HomeDribbling,...,AwayClearances,AwayIntercepts,AwayOffsides,AwayYellowCards,AwayRedCards,Away30mLineEntries,AwayPenaltyAreaEntries,AwayAttacks,AwayChanceBuildingRate,AwayPossession
10011354,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10011229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10012205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10011007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10012178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20340120,20.0,6.8,0.0,494.8,17.8,11.8,2.0,6.8,23.4,12.2,...,24.2,1.8,1.8,0.4,0.0,34.4,13.2,113.4,11.58,45.78
20340810,12.2,4.0,0.2,537.2,11.6,14.2,1.0,3.6,21.8,14.2,...,21.8,1.8,3.0,1.0,0.2,50.4,14.2,127.2,11.30,52.58
20343331,13.0,4.4,0.0,598.6,16.2,9.6,3.2,2.6,17.8,11.4,...,22.0,3.0,1.8,1.8,0.0,37.2,9.6,115.0,8.36,55.64
20342218,9.0,4.4,0.0,379.8,8.4,13.8,1.2,3.6,18.8,11.4,...,19.4,2.4,1.2,0.8,0.0,52.4,11.2,107.8,12.16,59.62


In [6]:
df_rate = pd.read_csv("./rating/all_years.csv", index_col=0, parse_dates=True)
df_rate

,kashima-antlers,urawa-red-diamonds,yokohama-fa-marinos,kawasaki-frontale,sanfrecce-hiroshima,vegalta-sendai,kashiwa-reysol,fc-tokyo,sagan-tosu,gamba-osaka,...,omiya-ardija,avispa-fukuoka,matsumoto-yamaga,oita-trinita,jef-united,tokyo-verdy,kyoto-sanga,tokushima-vortis,yokohama-fc,montedio-yamagata
2006-03-04,1500,1500,1500,1500,1500,1350,1350,1500,1350,1500,...,1500,1500,1350,1500,1500,1350,1500,1350,1350,1350
2006-03-05,1500,1500,1500,1500,1500,1350,1350,1500,1350,1500,...,1500,1500,1350,1500,1500,1350,1500,1350,1350,1350
2006-03-06,1508,1500,1508,1508,1492,1350,1350,1508,1350,1500,...,1508,1500,1350,1492,1492,1350,1492,1350,1350,1350
2006-03-07,1508,1500,1508,1508,1492,1350,1350,1508,1350,1500,...,1508,1500,1350,1492,1492,1350,1492,1350,1350,1350
2006-03-08,1508,1500,1508,1508,1492,1350,1351,1508,1350,1500,...,1508,1500,1350,1492,1492,1350,1492,1350,1351,1350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-16,1595,1490,1543,1678,1535,1411,1478,1541,1444,1577,...,1420,1331,1424,1473,1410,1408,1362,1313,1406,1372
2020-12-17,1595,1486,1543,1682,1526,1419,1487,1541,1455,1582,...,1420,1331,1424,1480,1410,1408,1362,1313,1402,1372
2020-12-18,1595,1486,1543,1682,1526,1419,1487,1541,1455,1582,...,1420,1331,1424,1480,1410,1408,1362,1313,1402,1372
2020-12-19,1595,1486,1543,1682,1526,1419,1487,1541,1455,1582,...,1420,1331,1424,1480,1410,1408,1362,1313,1402,1372


In [7]:
for index,row in df_match_allyears.iterrows():
    home_rate = df_rate.loc[row["Date"], row["Home"]]
    away_rate = df_rate.loc[row["Date"], row["Away"]]
    
    df_match_allyears.at[index,"HomeRate"] = home_rate
    df_match_allyears.at[index,"AwayRate"] = away_rate 

In [8]:
df_match_allyears

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,Stadium,Attendances,HomeRate,AwayRate
ID,,,,,,,,,,,,,,
6010903,2006,1,2006-03-04,gamba-osaka,urawa-red-diamonds,9,3,1,1,0,万博,20916,1500.0,1500.0
6012807,2006,1,2006-03-05,ventforet-kofu,shimizu-s-pulse,28,7,0,2,2,小瀬,14277,1500.0,1500.0
6012231,2006,1,2006-03-05,fc-tokyo,oita-trinita,22,31,2,0,1,味スタ,22531,1500.0,1500.0
6011323,2006,1,2006-03-05,jubilo-iwata,avispa-fukuoka,13,23,1,1,0,静岡,28564,1500.0,1500.0
6010820,2006,1,2006-03-05,nagoya-grampus-eight,cerezo-osaka,8,20,3,2,1,瑞穂陸,17199,1500.0,1500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20340120,2020,34,2020-12-19,kashima-antlers,cerezo-osaka,1,20,1,1,0,カシマ,11251,1595.0,1559.0
20340810,2020,34,2020-12-19,nagoya-grampus-eight,sanfrecce-hiroshima,8,10,1,0,1,豊田ス,16825,1514.0,1526.0
20343331,2020,34,2020-12-19,sagan-tosu,oita-trinita,33,31,2,2,0,駅スタ,8966,1455.0,1480.0


In [9]:
df_ranks= pd.read_csv("./ranks/all_years.csv",  index_col=0, parse_dates=True)
df_ranks

,kashima-antlers,urawa-red-diamonds,yokohama-fa-marinos,kawasaki-frontale,sanfrecce-hiroshima,vegalta-sendai,kashiwa-reysol,fc-tokyo,sagan-tosu,gamba-osaka,...,omiya-ardija,avispa-fukuoka,matsumoto-yamaga,oita-trinita,jef-united,tokyo-verdy,kyoto-sanga,tokushima-vortis,yokohama-fc,montedio-yamagata
2006-03-04,1,1,1,1,1,19,19,1,19,1,...,1,1,19,1,1,19,1,19,19,19
2006-03-05,3,1,3,3,3,19,19,3,19,1,...,3,3,19,3,3,19,3,19,19,19
2006-03-06,6,8,2,1,12,19,19,3,19,8,...,3,8,19,14,14,19,17,19,19,19
2006-03-07,6,8,2,1,12,19,19,3,19,8,...,3,8,19,14,14,19,17,19,19,19
2006-03-08,6,8,2,1,12,19,19,3,19,8,...,3,8,19,14,14,19,17,19,19,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-16,5,10,9,1,8,18,7,6,14,2,...,19,19,19,11,19,19,19,19,15,19
2020-12-17,5,10,9,1,8,16,7,6,14,2,...,19,19,19,11,19,19,19,19,15,19
2020-12-18,5,10,9,1,8,16,7,6,14,2,...,19,19,19,11,19,19,19,19,15,19
2020-12-19,5,10,9,1,8,16,7,6,14,2,...,19,19,19,11,19,19,19,19,15,19


In [10]:
for index,row in df_match_allyears.iterrows():
    home_rank = df_ranks.loc[row["Date"], row["Home"]]
    away_rank = df_ranks.loc[row["Date"], row["Away"]]
    
    df_match_allyears.at[index,"HomeRank"] = home_rank
    df_match_allyears.at[index,"AwayRank"] = away_rank

In [11]:
df_match_allyears

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,Stadium,Attendances,HomeRate,AwayRate,HomeRank,AwayRank
ID,,,,,,,,,,,,,,,,
6010903,2006,1,2006-03-04,gamba-osaka,urawa-red-diamonds,9,3,1,1,0,万博,20916,1500.0,1500.0,1.0,1.0
6012807,2006,1,2006-03-05,ventforet-kofu,shimizu-s-pulse,28,7,0,2,2,小瀬,14277,1500.0,1500.0,3.0,3.0
6012231,2006,1,2006-03-05,fc-tokyo,oita-trinita,22,31,2,0,1,味スタ,22531,1500.0,1500.0,3.0,3.0
6011323,2006,1,2006-03-05,jubilo-iwata,avispa-fukuoka,13,23,1,1,0,静岡,28564,1500.0,1500.0,3.0,3.0
6010820,2006,1,2006-03-05,nagoya-grampus-eight,cerezo-osaka,8,20,3,2,1,瑞穂陸,17199,1500.0,1500.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20340120,2020,34,2020-12-19,kashima-antlers,cerezo-osaka,1,20,1,1,0,カシマ,11251,1595.0,1559.0,5.0,4.0
20340810,2020,34,2020-12-19,nagoya-grampus-eight,sanfrecce-hiroshima,8,10,1,0,1,豊田ス,16825,1514.0,1526.0,3.0,8.0
20343331,2020,34,2020-12-19,sagan-tosu,oita-trinita,33,31,2,2,0,駅スタ,8966,1455.0,1480.0,14.0,11.0


In [12]:
df_points = pd.read_csv("./points/all_years.csv" ,index_col=0, parse_dates=True)
df_points

,kashima-antlers,urawa-red-diamonds,yokohama-fa-marinos,kawasaki-frontale,sanfrecce-hiroshima,vegalta-sendai,kashiwa-reysol,fc-tokyo,sagan-tosu,gamba-osaka,...,omiya-ardija,avispa-fukuoka,matsumoto-yamaga,oita-trinita,jef-united,tokyo-verdy,kyoto-sanga,tokushima-vortis,yokohama-fc,montedio-yamagata
date,,,,,,,,,,,,,,,,,,,,,
2006-03-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2006-03-05,0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2006-03-06,3,1,3,3,0,0,0,3,0,1,...,3,1,0,0,0,0,0,0,0,0
2006-03-07,3,1,3,3,0,0,0,3,0,1,...,3,1,0,0,0,0,0,0,0,0
2006-03-08,3,1,3,3,0,0,0,3,0,1,...,3,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-16,58,46,47,77,48,24,49,54,32,62,...,0,0,0,39,0,0,0,0,30,0
2020-12-17,58,46,47,80,48,27,52,54,35,65,...,0,0,0,42,0,0,0,0,30,0
2020-12-18,58,46,47,80,48,27,52,54,35,65,...,0,0,0,42,0,0,0,0,30,0


In [13]:
for index,row in df_match_allyears.iterrows():
    home_points = df_points.loc[row["Date"], row["Home"]]
    away_points = df_points.loc[row["Date"], row["Away"]]
    
    df_match_allyears.at[index,"HomePoints"] = home_points
    df_match_allyears.at[index,"AwayPoints"] = away_points

In [14]:
df_match_allyears

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,Stadium,Attendances,HomeRate,AwayRate,HomeRank,AwayRank,HomePoints,AwayPoints
ID,,,,,,,,,,,,,,,,,,
6010903,2006,1,2006-03-04,gamba-osaka,urawa-red-diamonds,9,3,1,1,0,万博,20916,1500.0,1500.0,1.0,1.0,0.0,0.0
6012807,2006,1,2006-03-05,ventforet-kofu,shimizu-s-pulse,28,7,0,2,2,小瀬,14277,1500.0,1500.0,3.0,3.0,0.0,0.0
6012231,2006,1,2006-03-05,fc-tokyo,oita-trinita,22,31,2,0,1,味スタ,22531,1500.0,1500.0,3.0,3.0,0.0,0.0
6011323,2006,1,2006-03-05,jubilo-iwata,avispa-fukuoka,13,23,1,1,0,静岡,28564,1500.0,1500.0,3.0,3.0,0.0,0.0
6010820,2006,1,2006-03-05,nagoya-grampus-eight,cerezo-osaka,8,20,3,2,1,瑞穂陸,17199,1500.0,1500.0,3.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20340120,2020,34,2020-12-19,kashima-antlers,cerezo-osaka,1,20,1,1,0,カシマ,11251,1595.0,1559.0,5.0,4.0,58.0,59.0
20340810,2020,34,2020-12-19,nagoya-grampus-eight,sanfrecce-hiroshima,8,10,1,0,1,豊田ス,16825,1514.0,1526.0,3.0,8.0,60.0,48.0
20343331,2020,34,2020-12-19,sagan-tosu,oita-trinita,33,31,2,2,0,駅スタ,8966,1455.0,1480.0,14.0,11.0,35.0,42.0


In [15]:
df_gd = pd.read_csv("./goal_differences/all_years.csv",index_col=0, parse_dates=True)
df_gd

,kashima-antlers,urawa-red-diamonds,yokohama-fa-marinos,kawasaki-frontale,sanfrecce-hiroshima,vegalta-sendai,kashiwa-reysol,fc-tokyo,sagan-tosu,gamba-osaka,...,omiya-ardija,avispa-fukuoka,matsumoto-yamaga,oita-trinita,jef-united,tokyo-verdy,kyoto-sanga,tokushima-vortis,yokohama-fc,montedio-yamagata
date,,,,,,,,,,,,,,,,,,,,,
2006-03-04,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2006-03-05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2006-03-06,1,0,3,6,-1,0,0,2,0,0,...,2,0,0,-2,-2,0,-3,0,0,0
2006-03-07,1,0,3,6,-1,0,0,2,0,0,...,2,0,0,-2,-2,0,-3,0,0,0
2006-03-08,1,0,3,6,-1,0,0,2,0,0,...,2,0,0,-2,-2,0,-3,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-16,11,-9,12,54,11,-26,14,4,-7,4,...,0,0,0,-10,0,0,0,0,-22,0
2020-12-17,11,-11,12,56,10,-25,15,4,-6,6,...,0,0,0,-9,0,0,0,0,-24,0
2020-12-18,11,-11,12,56,10,-25,15,4,-6,6,...,0,0,0,-9,0,0,0,0,-24,0


In [16]:
for index,row in df_match_allyears.iterrows():
    home_gd = df_gd.loc[row["Date"], row["Home"]]
    away_gd = df_gd.loc[row["Date"], row["Away"]]
    
    df_match_allyears.at[index,"HomeGD"] = home_gd
    df_match_allyears.at[index,"AwayGD"] = away_gd

In [17]:
df_match_allyears

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,Stadium,Attendances,HomeRate,AwayRate,HomeRank,AwayRank,HomePoints,AwayPoints,HomeGD,AwayGD
ID,,,,,,,,,,,,,,,,,,,,
6010903,2006,1,2006-03-04,gamba-osaka,urawa-red-diamonds,9,3,1,1,0,万博,20916,1500.0,1500.0,1.0,1.0,0.0,0.0,0.0,0.0
6012807,2006,1,2006-03-05,ventforet-kofu,shimizu-s-pulse,28,7,0,2,2,小瀬,14277,1500.0,1500.0,3.0,3.0,0.0,0.0,0.0,0.0
6012231,2006,1,2006-03-05,fc-tokyo,oita-trinita,22,31,2,0,1,味スタ,22531,1500.0,1500.0,3.0,3.0,0.0,0.0,0.0,0.0
6011323,2006,1,2006-03-05,jubilo-iwata,avispa-fukuoka,13,23,1,1,0,静岡,28564,1500.0,1500.0,3.0,3.0,0.0,0.0,0.0,0.0
6010820,2006,1,2006-03-05,nagoya-grampus-eight,cerezo-osaka,8,20,3,2,1,瑞穂陸,17199,1500.0,1500.0,3.0,3.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20340120,2020,34,2020-12-19,kashima-antlers,cerezo-osaka,1,20,1,1,0,カシマ,11251,1595.0,1559.0,5.0,4.0,58.0,59.0,11.0,9.0
20340810,2020,34,2020-12-19,nagoya-grampus-eight,sanfrecce-hiroshima,8,10,1,0,1,豊田ス,16825,1514.0,1526.0,3.0,8.0,60.0,48.0,16.0,10.0
20343331,2020,34,2020-12-19,sagan-tosu,oita-trinita,33,31,2,2,0,駅スタ,8966,1455.0,1480.0,14.0,11.0,35.0,42.0,-6.0,-9.0


In [18]:
df_allyears = pd.concat([df_match_allyears,df_recent_stats],axis=1)
df_allyears

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,...,AwayClearances,AwayIntercepts,AwayOffsides,AwayYellowCards,AwayRedCards,Away30mLineEntries,AwayPenaltyAreaEntries,AwayAttacks,AwayChanceBuildingRate,AwayPossession
6010524,2006,1,2006-03-05,yokohama-fa-marinos,kyoto-sanga,5,24,4,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6010820,2006,1,2006-03-05,nagoya-grampus-eight,cerezo-osaka,8,20,3,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6010903,2006,1,2006-03-04,gamba-osaka,urawa-red-diamonds,9,3,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6011001,2006,1,2006-03-05,sanfrecce-hiroshima,kashima-antlers,10,1,3,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6011323,2006,1,2006-03-05,jubilo-iwata,avispa-fukuoka,13,23,1,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20341121,2020,34,2020-12-19,kashiwa-reysol,kawasaki-frontale,11,21,2,3,2,...,19.0,2.4,2.2,0.0,0.2,64.0,24.4,115.2,16.08,54.30
20342218,2020,34,2020-12-19,fc-tokyo,vissel-kobe,22,18,1,0,1,...,19.4,2.4,1.2,0.8,0.0,52.4,11.2,107.8,12.16,59.62
20343331,2020,34,2020-12-19,sagan-tosu,oita-trinita,33,31,2,2,0,...,22.0,3.0,1.8,1.8,0.0,37.2,9.6,115.0,8.36,55.64
20343405,2020,34,2020-12-19,yokohama-fc,yokohama-fa-marinos,34,5,3,1,1,...,23.2,2.0,2.4,2.0,0.2,61.2,19.6,124.4,11.58,54.54


In [19]:
df = df_allyears[df_allyears["Year"] >= 2012]
df

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,...,AwayClearances,AwayIntercepts,AwayOffsides,AwayYellowCards,AwayRedCards,Away30mLineEntries,AwayPenaltyAreaEntries,AwayAttacks,AwayChanceBuildingRate,AwayPossession
12010807,2012,1,2012-03-10,nagoya-grampus-eight,shimizu-s-pulse,8,7,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12010918,2012,1,2012-03-10,gamba-osaka,vissel-kobe,9,18,2,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12011003,2012,1,2012-03-10,sanfrecce-hiroshima,urawa-red-diamonds,10,3,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12011105,2012,1,2012-03-11,kashiwa-reysol,yokohama-fa-marinos,11,5,3,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12011413,2012,1,2012-03-10,consadole-sapporo,jubilo-iwata,14,13,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20341121,2020,34,2020-12-19,kashiwa-reysol,kawasaki-frontale,11,21,2,3,2,...,19.0,2.4,2.2,0.0,0.2,64.0,24.4,115.2,16.08,54.30
20342218,2020,34,2020-12-19,fc-tokyo,vissel-kobe,22,18,1,0,1,...,19.4,2.4,1.2,0.8,0.0,52.4,11.2,107.8,12.16,59.62
20343331,2020,34,2020-12-19,sagan-tosu,oita-trinita,33,31,2,2,0,...,22.0,3.0,1.8,1.8,0.0,37.2,9.6,115.0,8.36,55.64
20343405,2020,34,2020-12-19,yokohama-fc,yokohama-fa-marinos,34,5,3,1,1,...,23.2,2.0,2.4,2.0,0.2,61.2,19.6,124.4,11.58,54.54


### statsの欠損値を前の年の降格チームの平均値で埋める

In [20]:
df_clubs = pd.read_csv("./clubs.csv", index_col=0)
df_clubs

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
2006,kashima-antlers,jef-united,urawa-red-diamonds,yokohama-fa-marinos,shimizu-s-pulse,nagoya-grampus-eight,gamba-osaka,sanfrecce-hiroshima,jubilo-iwata,cerezo-osaka,kawasaki-frontale,fc-tokyo,avispa-fukuoka,kyoto-sanga,omiya-ardija,ventforet-kofu,oita-trinita,albirex-niigata
2007,kashima-antlers,jef-united,urawa-red-diamonds,yokohama-fa-marinos,shimizu-s-pulse,nagoya-grampus-eight,gamba-osaka,sanfrecce-hiroshima,jubilo-iwata,kawasaki-frontale,fc-tokyo,omiya-ardija,ventforet-kofu,oita-trinita,albirex-niigata,vissel-kobe,kashiwa-reysol,yokohama-fc
2008,kashima-antlers,jef-united,urawa-red-diamonds,yokohama-fa-marinos,shimizu-s-pulse,nagoya-grampus-eight,gamba-osaka,jubilo-iwata,kawasaki-frontale,fc-tokyo,omiya-ardija,oita-trinita,albirex-niigata,vissel-kobe,kashiwa-reysol,consadole-sapporo,tokyo-verdy,kyoto-sanga
2009,kashima-antlers,jef-united,urawa-red-diamonds,yokohama-fa-marinos,shimizu-s-pulse,nagoya-grampus-eight,gamba-osaka,jubilo-iwata,kawasaki-frontale,fc-tokyo,omiya-ardija,oita-trinita,albirex-niigata,vissel-kobe,kashiwa-reysol,kyoto-sanga,sanfrecce-hiroshima,montedio-yamagata
2010,kashima-antlers,urawa-red-diamonds,yokohama-fa-marinos,shimizu-s-pulse,nagoya-grampus-eight,gamba-osaka,jubilo-iwata,kawasaki-frontale,fc-tokyo,omiya-ardija,albirex-niigata,vissel-kobe,kyoto-sanga,sanfrecce-hiroshima,montedio-yamagata,vegalta-sendai,cerezo-osaka,shonan-bellmare
2011,kashima-antlers,urawa-red-diamonds,yokohama-fa-marinos,shimizu-s-pulse,nagoya-grampus-eight,gamba-osaka,jubilo-iwata,kawasaki-frontale,omiya-ardija,albirex-niigata,vissel-kobe,sanfrecce-hiroshima,montedio-yamagata,vegalta-sendai,cerezo-osaka,kashiwa-reysol,ventforet-kofu,avispa-fukuoka
2012,kashima-antlers,urawa-red-diamonds,yokohama-fa-marinos,shimizu-s-pulse,nagoya-grampus-eight,gamba-osaka,jubilo-iwata,kawasaki-frontale,omiya-ardija,albirex-niigata,vissel-kobe,sanfrecce-hiroshima,vegalta-sendai,cerezo-osaka,kashiwa-reysol,fc-tokyo,sagan-tosu,consadole-sapporo
2013,kashima-antlers,urawa-red-diamonds,yokohama-fa-marinos,shimizu-s-pulse,nagoya-grampus-eight,jubilo-iwata,kawasaki-frontale,omiya-ardija,albirex-niigata,sanfrecce-hiroshima,vegalta-sendai,cerezo-osaka,kashiwa-reysol,fc-tokyo,sagan-tosu,ventforet-kofu,shonan-bellmare,oita-trinita
2014,kashima-antlers,urawa-red-diamonds,yokohama-fa-marinos,shimizu-s-pulse,nagoya-grampus-eight,kawasaki-frontale,omiya-ardija,albirex-niigata,sanfrecce-hiroshima,vegalta-sendai,cerezo-osaka,kashiwa-reysol,fc-tokyo,sagan-tosu,ventforet-kofu,gamba-osaka,vissel-kobe,tokushima-vortis
2015,kashima-antlers,urawa-red-diamonds,yokohama-fa-marinos,shimizu-s-pulse,nagoya-grampus-eight,kawasaki-frontale,albirex-niigata,sanfrecce-hiroshima,vegalta-sendai,kashiwa-reysol,fc-tokyo,sagan-tosu,ventforet-kofu,gamba-osaka,vissel-kobe,shonan-bellmare,matsumoto-yamaga,montedio-yamagata


In [21]:
for year in range(2012,2021):
    relegation_clubs = list(set(df_clubs.loc[year-1])-set(df_clubs.loc[year]))
    
    df_relegation_stats = pd.DataFrame(columns=df_recent_stats.columns)

    for relegation_club in  relegation_clubs:
        relegation_stats     = (
            df[
                (df["Year"] == year-1) &
                ((df["Home"] == relegation_club)|(df["Away"] == relegation_club))
            ][df_recent_stats.columns].mean())

        df_relegation_stats.loc[relegation_club] = relegation_stats
        
        relegation_stats_mean = df_relegation_stats.mean()
    
    df[df["Year"] == year] = df[df["Year"] == year].fillna(relegation_stats_mean)



<ipython-input-21-1854c66c225e>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df["Year"] == year] = df[df["Year"] == year].fillna(relegation_stats_mean)
/home/yokoyama/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1717: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)
<ipython-input-21-1854c66c225e>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [22]:
df

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,...,AwayClearances,AwayIntercepts,AwayOffsides,AwayYellowCards,AwayRedCards,Away30mLineEntries,AwayPenaltyAreaEntries,AwayAttacks,AwayChanceBuildingRate,AwayPossession
12010807,2012,1,2012-03-10,nagoya-grampus-eight,shimizu-s-pulse,8,7,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12010918,2012,1,2012-03-10,gamba-osaka,vissel-kobe,9,18,2,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12011003,2012,1,2012-03-10,sanfrecce-hiroshima,urawa-red-diamonds,10,3,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12011105,2012,1,2012-03-11,kashiwa-reysol,yokohama-fa-marinos,11,5,3,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12011413,2012,1,2012-03-10,consadole-sapporo,jubilo-iwata,14,13,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20341121,2020,34,2020-12-19,kashiwa-reysol,kawasaki-frontale,11,21,2,3,2,...,19.0,2.4,2.2,0.0,0.2,64.0,24.4,115.2,16.08,54.30
20342218,2020,34,2020-12-19,fc-tokyo,vissel-kobe,22,18,1,0,1,...,19.4,2.4,1.2,0.8,0.0,52.4,11.2,107.8,12.16,59.62
20343331,2020,34,2020-12-19,sagan-tosu,oita-trinita,33,31,2,2,0,...,22.0,3.0,1.8,1.8,0.0,37.2,9.6,115.0,8.36,55.64
20343405,2020,34,2020-12-19,yokohama-fc,yokohama-fa-marinos,34,5,3,1,1,...,23.2,2.0,2.4,2.0,0.2,61.2,19.6,124.4,11.58,54.54


In [23]:
df.columns

Index(['Year', 'Sec', 'Date', 'Home', 'Away', 'HomeID', 'AwayID', 'HomeGF',
       'AwayGF', 'W/L', 'Stadium', 'Attendances', 'HomeRate', 'AwayRate',
       'HomeRank', 'AwayRank', 'HomePoints', 'AwayPoints', 'HomeGD', 'AwayGD',
       'HomeShots', 'HomeShotsOnTarget', 'HomeShotsFromPK', 'HomePasses',
       'HomeCrosses', 'HomeDirectFK', 'HomeIndirectFK', 'HomeCK',
       'HomeThrowin', 'HomeDribbling', 'HomeTackles', 'HomeClearances',
       'HomeIntercepts', 'HomeOffsides', 'HomeYellowCards', 'HomeRedCards',
       'Home30mLineEntries', 'HomePenaltyAreaEntries', 'HomeAttacks',
       'HomeChanceBuildingRate', 'HomePossession', 'AwayShots',
       'AwayShotsOnTarget', 'AwayShotsFromPK', 'AwayPasses', 'AwayCrosses',
       'AwayDirectFK', 'AwayIndirectFK', 'AwayCK', 'AwayThrowin',
       'AwayDribbling', 'AwayTackles', 'AwayClearances', 'AwayIntercepts',
       'AwayOffsides', 'AwayYellowCards', 'AwayRedCards', 'Away30mLineEntries',
       'AwayPenaltyAreaEntries', 'AwayAttacks', 'Awa

### 予測

In [24]:
X_features = [ 
    'Sec', 'HomeRate', 'AwayRate','HomeRank', 'AwayRank', 'HomePoints', 'AwayPoints', 'HomeGD', 'AwayGD','HomeShots', 'HomeShotsOnTarget', 'HomeShotsFromPK', 'HomePasses',
    'HomeCrosses', 'HomeDirectFK', 'HomeIndirectFK', 'HomeCK','HomeThrowin', 'HomeDribbling', 'HomeTackles', 'HomeClearances',
    'HomeIntercepts', 'HomeOffsides', 'HomeYellowCards', 'HomeRedCards','Home30mLineEntries', 'HomePenaltyAreaEntries', 'HomeAttacks',
    'HomeChanceBuildingRate', 'HomePossession', 'AwayShots','AwayShotsOnTarget', 'AwayShotsFromPK', 'AwayPasses', 'AwayCrosses',
    'AwayDirectFK', 'AwayIndirectFK', 'AwayCK', 'AwayThrowin','AwayDribbling', 'AwayTackles', 'AwayClearances', 'AwayIntercepts',
    'AwayOffsides', 'AwayYellowCards', 'AwayRedCards', 'Away30mLineEntries','AwayPenaltyAreaEntries', 
    'AwayAttacks', 'AwayChanceBuildingRate','AwayPossession'
]

accuracies = []
loglosses = []

feature_imps = []

for  year in range(2015, 2021):
    
    print(f"=={year}=================================")
    
    X_train = df[(df["Year"] == (year-3)) | (df["Year"] == (year-2)) | (df["Year"] == (year-1))].dropna(how="any")[X_features].values
    y_train = df[(df["Year"] == (year-3)) | (df["Year"] == (year-2)) | (df["Year"] == (year-1))].dropna(how="any")["W/L"].values
    
    X_test = df[df["Year"] == year][X_features].values
    y_test = df[df["Year"] == year]["W/L"].values
    
    clf = RandomForestClassifier(random_state=1234, max_depth=6)
    clf.fit(X_train, y_train)
    
    y_pred_proba = clf.predict_proba(X_test)
    
    accuracy = clf.score(X_test, y_test)
    accuracies.append(accuracy)
    
    feature_imps.append( clf.feature_importances_.tolist())
    
    logloss = log_loss(y_test,y_pred_proba,labels=[0,1,2])
    loglosses.append(logloss)
    
    print(f"Accuracy : {accuracy}")
    print(f"Log loss : {logloss}")
    print(" ")
    
print(" ")
print("======================================")    
print(f"的中率(Accuracy) : {np.mean(accuracies)}" )
print(f"対数損失(Log loss) : {np.mean(loglosses)}" )

==2015=================================
Accuracy : 0.46405228758169936
Log loss : 1.04933353293205
 
==2016=================================
Accuracy : 0.5065359477124183
Log loss : 1.0209660287238593
 
==2017=================================
Accuracy : 0.4738562091503268
Log loss : 1.0507756047601946
 
==2018=================================
Accuracy : 0.4084967320261438
Log loss : 1.074191718090266
 
==2019=================================
Accuracy : 0.45098039215686275
Log loss : 1.0670393739429698
 
==2020=================================
Accuracy : 0.49019607843137253
Log loss : 1.0348221444127843
 
 
的中率(Accuracy) : 0.465686274509804
対数損失(Log loss) : 1.0495214004770206


In [25]:
feature_imps

[[0.01600088987720968,
  0.031190466441490435,
  0.03595238223416291,
  0.033203512727439786,
  0.0196494383030003,
  0.031025802500027542,
  0.014210132648284319,
  0.02562191869724923,
  0.021550734429692377,
  0.020651907432831018,
  0.016495381139660162,
  0.003543690279470303,
  0.02442111025354626,
  0.020328674237602547,
  0.02198963385256183,
  0.026855539401310077,
  0.017124365473011566,
  0.021522510252026342,
  0.014165019643680667,
  0.022655683983536175,
  0.02949607187359088,
  0.009373878949706007,
  0.021339423559645705,
  0.007523351324775033,
  0.003140084843571203,
  0.020311532520467426,
  0.02538078889843271,
  0.018284522521116345,
  0.0366285754681276,
  0.03363263210082062,
  0.02766114844050541,
  0.01623355865989055,
  0.0048818127482362994,
  0.022315941442183886,
  0.019534034410179375,
  0.014140120238888828,
  0.017534163003507627,
  0.014505845909720809,
  0.014874471877903243,
  0.016163943801986474,
  0.016025363236982882,
  0.014903787739239956,
  0.0

In [26]:
np.array(feature_imps)

array([[0.01600089, 0.03119047, 0.03595238, 0.03320351, 0.01964944,
        0.0310258 , 0.01421013, 0.02562192, 0.02155073, 0.02065191,
        0.01649538, 0.00354369, 0.02442111, 0.02032867, 0.02198963,
        0.02685554, 0.01712437, 0.02152251, 0.01416502, 0.02265568,
        0.02949607, 0.00937388, 0.02133942, 0.00752335, 0.00314008,
        0.02031153, 0.02538079, 0.01828452, 0.03662858, 0.03363263,
        0.02766115, 0.01623356, 0.00488181, 0.02231594, 0.01953403,
        0.01414012, 0.01753416, 0.01450585, 0.01487447, 0.01616394,
        0.01602536, 0.01490379, 0.0223358 , 0.02095424, 0.01146934,
        0.00322625, 0.01762157, 0.02399714, 0.01453009, 0.02168565,
        0.02213608],
       [0.01662691, 0.05025546, 0.02792031, 0.02245595, 0.02025595,
        0.02802513, 0.02255554, 0.02325362, 0.02452415, 0.03120629,
        0.01452187, 0.00240254, 0.0331513 , 0.01555285, 0.01544606,
        0.02019349, 0.01638836, 0.02353001, 0.0192005 , 0.01729833,
        0.02609483, 0.01546

In [27]:
np.mean(np.array(feature_imps),axis=0)

array([0.01523221, 0.03916491, 0.03143762, 0.02008711, 0.02144123,
       0.02385098, 0.01706637, 0.02552724, 0.02263827, 0.02275924,
       0.01699375, 0.00315091, 0.0329488 , 0.02013144, 0.01634653,
       0.01767808, 0.0162698 , 0.02010284, 0.01971693, 0.01743997,
       0.02238009, 0.01283889, 0.01637628, 0.01105171, 0.00279927,
       0.03042772, 0.02688464, 0.02078071, 0.02782884, 0.02746993,
       0.02145578, 0.02072945, 0.00557549, 0.03186114, 0.0192516 ,
       0.01898379, 0.01563612, 0.01657785, 0.01540306, 0.01825031,
       0.01615189, 0.01971373, 0.01901529, 0.01588426, 0.00999113,
       0.00323826, 0.02779692, 0.0186499 , 0.01655927, 0.02286685,
       0.0275856 ])

In [28]:
pd.DataFrame(np.mean(np.array(feature_imps),axis=0), index = X_features).sort_values(0,ascending = False).head(20)

,0
HomeRate,0.039165
HomePasses,0.032949
AwayPasses,0.031861
AwayRate,0.031438
Home30mLineEntries,0.030428
HomeChanceBuildingRate,0.027829
Away30mLineEntries,0.027797
AwayPossession,0.027586
HomePossession,0.027470
HomePenaltyAreaEntries,0.026885


### StatsのHomeとAwayの差分を食わせる

In [29]:
df.columns

Index(['Year', 'Sec', 'Date', 'Home', 'Away', 'HomeID', 'AwayID', 'HomeGF',
       'AwayGF', 'W/L', 'Stadium', 'Attendances', 'HomeRate', 'AwayRate',
       'HomeRank', 'AwayRank', 'HomePoints', 'AwayPoints', 'HomeGD', 'AwayGD',
       'HomeShots', 'HomeShotsOnTarget', 'HomeShotsFromPK', 'HomePasses',
       'HomeCrosses', 'HomeDirectFK', 'HomeIndirectFK', 'HomeCK',
       'HomeThrowin', 'HomeDribbling', 'HomeTackles', 'HomeClearances',
       'HomeIntercepts', 'HomeOffsides', 'HomeYellowCards', 'HomeRedCards',
       'Home30mLineEntries', 'HomePenaltyAreaEntries', 'HomeAttacks',
       'HomeChanceBuildingRate', 'HomePossession', 'AwayShots',
       'AwayShotsOnTarget', 'AwayShotsFromPK', 'AwayPasses', 'AwayCrosses',
       'AwayDirectFK', 'AwayIndirectFK', 'AwayCK', 'AwayThrowin',
       'AwayDribbling', 'AwayTackles', 'AwayClearances', 'AwayIntercepts',
       'AwayOffsides', 'AwayYellowCards', 'AwayRedCards', 'Away30mLineEntries',
       'AwayPenaltyAreaEntries', 'AwayAttacks', 'Awa

In [30]:
stats_features = [
    'Rate','Rank','Points','GD','Shots', 'ShotsOnTarget', 'ShotsFromPK', 'Passes','Crosses', 'DirectFK', 'IndirectFK',
    'CK','Throwin', 'Dribbling','Tackles', 'Clearances','Intercepts', 'Offsides', 'YellowCards', 'RedCards',
    '30mLineEntries', 'PenaltyAreaEntries', 'Attacks','ChanceBuildingRate', 'Possession'
]

In [31]:
for fet in stats_features:
    df.insert(len(df.columns), f"{fet}Diff", df[f"Home{fet}"] - df[f"Away{fet}"])

In [32]:
df

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,...,ClearancesDiff,InterceptsDiff,OffsidesDiff,YellowCardsDiff,RedCardsDiff,30mLineEntriesDiff,PenaltyAreaEntriesDiff,AttacksDiff,ChanceBuildingRateDiff,PossessionDiff
12010807,2012,1,2012-03-10,nagoya-grampus-eight,shimizu-s-pulse,8,7,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12010918,2012,1,2012-03-10,gamba-osaka,vissel-kobe,9,18,2,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12011003,2012,1,2012-03-10,sanfrecce-hiroshima,urawa-red-diamonds,10,3,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12011105,2012,1,2012-03-11,kashiwa-reysol,yokohama-fa-marinos,11,5,3,3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12011413,2012,1,2012-03-10,consadole-sapporo,jubilo-iwata,14,13,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20341121,2020,34,2020-12-19,kashiwa-reysol,kawasaki-frontale,11,21,2,3,2,...,9.0,0.4,-0.2,1.0,-0.2,-24.2,-12.2,8.6,-6.96,-7.24
20342218,2020,34,2020-12-19,fc-tokyo,vissel-kobe,22,18,1,0,1,...,7.4,-1.8,-0.8,0.0,0.0,-26.6,-1.4,8.6,-4.42,-16.82
20343331,2020,34,2020-12-19,sagan-tosu,oita-trinita,33,31,2,2,0,...,4.0,-0.4,1.2,-1.0,0.0,14.8,5.4,1.6,2.80,-2.00
20343405,2020,34,2020-12-19,yokohama-fc,yokohama-fa-marinos,34,5,3,1,1,...,-5.2,-0.6,0.0,-1.6,-0.2,-10.0,-8.2,-17.6,0.92,1.76


In [33]:
df.columns

Index(['Year', 'Sec', 'Date', 'Home', 'Away', 'HomeID', 'AwayID', 'HomeGF',
       'AwayGF', 'W/L', 'Stadium', 'Attendances', 'HomeRate', 'AwayRate',
       'HomeRank', 'AwayRank', 'HomePoints', 'AwayPoints', 'HomeGD', 'AwayGD',
       'HomeShots', 'HomeShotsOnTarget', 'HomeShotsFromPK', 'HomePasses',
       'HomeCrosses', 'HomeDirectFK', 'HomeIndirectFK', 'HomeCK',
       'HomeThrowin', 'HomeDribbling', 'HomeTackles', 'HomeClearances',
       'HomeIntercepts', 'HomeOffsides', 'HomeYellowCards', 'HomeRedCards',
       'Home30mLineEntries', 'HomePenaltyAreaEntries', 'HomeAttacks',
       'HomeChanceBuildingRate', 'HomePossession', 'AwayShots',
       'AwayShotsOnTarget', 'AwayShotsFromPK', 'AwayPasses', 'AwayCrosses',
       'AwayDirectFK', 'AwayIndirectFK', 'AwayCK', 'AwayThrowin',
       'AwayDribbling', 'AwayTackles', 'AwayClearances', 'AwayIntercepts',
       'AwayOffsides', 'AwayYellowCards', 'AwayRedCards', 'Away30mLineEntries',
       'AwayPenaltyAreaEntries', 'AwayAttacks', 'Awa

In [34]:
X_features = [ 
    'Sec', 'HomeRate', 'AwayRate','HomeRank', 'AwayRank', 'HomePoints', 'AwayPoints', 
    'HomeGD', 'AwayGD','HomeShots', 'HomeShotsOnTarget', 'HomeShotsFromPK', 'HomePasses',
    'HomeCrosses', 'HomeDirectFK', 'HomeIndirectFK', 'HomeCK','HomeThrowin', 'HomeDribbling', 'HomeTackles', 'HomeClearances',
    'HomeIntercepts', 'HomeOffsides', 'HomeYellowCards', 'HomeRedCards','Home30mLineEntries', 'HomePenaltyAreaEntries', 'HomeAttacks',
    'HomeChanceBuildingRate', 'HomePossession', 'AwayShots','AwayShotsOnTarget', 'AwayShotsFromPK', 'AwayPasses', 'AwayCrosses',
    'AwayDirectFK', 'AwayIndirectFK', 'AwayCK', 'AwayThrowin','AwayDribbling', 'AwayTackles', 'AwayClearances', 'AwayIntercepts',
    'AwayOffsides', 'AwayYellowCards', 'AwayRedCards', 'Away30mLineEntries','AwayPenaltyAreaEntries', 
    'AwayAttacks', 'AwayChanceBuildingRate','AwayPossession',
    'RateDiff',  'RankDiff', 'PointsDiff', 'GDDiff','ShotsDiff', 'ShotsOnTargetDiff',
    'ShotsFromPKDiff', 'PassesDiff', 'CrossesDiff', 'DirectFKDiff','IndirectFKDiff', 
    'CKDiff', 'ThrowinDiff', 'DribblingDiff','TacklesDiff', 'ClearancesDiff', 'InterceptsDiff', 'OffsidesDiff','YellowCardsDiff', 
    'RedCardsDiff', '30mLineEntriesDiff','PenaltyAreaEntriesDiff', 'AttacksDiff', 'ChanceBuildingRateDiff','PossessionDiff'
]

accuracies = []
loglosses = []
preds = []
tests = []

feature_imps = []


for  year in range(2015, 2021):
    
    print(f"=={year}=================================")
    
    X_train = df[(df["Year"] == (year-3)) | (df["Year"] == (year-2)) | (df["Year"] == (year-1))].dropna(how="any")[X_features].values
    y_train = df[(df["Year"] == (year-3)) | (df["Year"] == (year-2)) | (df["Year"] == (year-1))].dropna(how="any")["W/L"].values
    
    X_test = df[df["Year"] == year][X_features].values
    y_test = df[df["Year"] == year]["W/L"].values
    
    clf = RandomForestClassifier(random_state=1234, max_depth=7)
    clf.fit(X_train, y_train)
    
    y_pred_proba = clf.predict_proba(X_test)
    y_pred = clf.predict(X_test)
    preds.extend(y_pred.tolist())
    
    tests.extend(y_test.tolist())
    
    accuracy = clf.score(X_test, y_test)
    accuracies.append(accuracy)
    
    feature_imps.append( clf.feature_importances_.tolist())
    
    logloss = log_loss(y_test,y_pred_proba,labels=[0,1,2])
    loglosses.append(logloss)
    
    print(f"Accuracy : {accuracy}")
    print(f"Log loss : {logloss}")
    print(" ")
    
    
print(" ")
print("======================================")    
print(f"的中率(Accuracy) : {np.mean(accuracies)}" )
print(f"対数損失(Log loss) : {np.mean(loglosses)}" )

==2015=================================
Accuracy : 0.49019607843137253
Log loss : 1.0341498904915338
 
==2016=================================
Accuracy : 0.5261437908496732
Log loss : 1.028375196974425
 
==2017=================================
Accuracy : 0.4542483660130719
Log loss : 1.0484404139964787
 
==2018=================================
Accuracy : 0.4084967320261438
Log loss : 1.06775031280423
 
==2019=================================
Accuracy : 0.4411764705882353
Log loss : 1.0730882687188155
 
==2020=================================
Accuracy : 0.5228758169934641
Log loss : 1.0262695109154236
 
 
的中率(Accuracy) : 0.4738562091503267
対数損失(Log loss) : 1.0463455989834844


In [35]:
pd.DataFrame(np.mean(np.array(feature_imps),axis=0), index = X_features).sort_values(0,ascending = False).head(10)

,0
RateDiff,0.033917
HomeRate,0.022697
30mLineEntriesDiff,0.022035
PassesDiff,0.021784
ChanceBuildingRateDiff,0.020333
Home30mLineEntries,0.019393
PossessionDiff,0.019172
PointsDiff,0.018692
Away30mLineEntries,0.018452
AwayPasses,0.017351


In [36]:
import collections
collections.Counter(preds)

Counter({2: 778, 1: 1046, 0: 12})

In [37]:
collections.Counter(tests)

Counter({2: 674, 0: 427, 1: 735})

In [38]:
correct_preds = []
for t,p in zip(tests,preds):
    if t==p:
        correct_preds.append(p)

In [39]:
collections.Counter(correct_preds)

Counter({2: 364, 1: 502, 0: 4})

In [40]:
len(correct_preds)

870

In [41]:
collections.Counter(tests)

Counter({2: 674, 0: 427, 1: 735})

In [42]:
502/735,364/674,4/427

(0.6829931972789116, 0.5400593471810089, 0.00936768149882904)

In [43]:
870/1836

0.4738562091503268

In [44]:
pd.DataFrame(clf.feature_importances_, index = X_features).sort_values(0,ascending = False).head(20)

,0
RateDiff,0.026722
30mLineEntriesDiff,0.023771
AwayPasses,0.023655
Home30mLineEntries,0.022162
Away30mLineEntries,0.020633
GDDiff,0.019469
PenaltyAreaEntriesDiff,0.018905
AwayRate,0.018509
PassesDiff,0.018451
AwayShotsOnTarget,0.017987


### RFとポアソン分布の予測 比較

In [45]:
len(y_pred)

306

In [46]:
print(preds)

[2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 0, 2, 1, 2, 2, 1, 1, 2, 2, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 0, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 0, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 

In [47]:
df_analysis= df[df["Date"].dt.year >= 2015]
df_analysis

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,...,ClearancesDiff,InterceptsDiff,OffsidesDiff,YellowCardsDiff,RedCardsDiff,30mLineEntriesDiff,PenaltyAreaEntriesDiff,AttacksDiff,ChanceBuildingRateDiff,PossessionDiff
15010521,2015,1,2015-03-07,yokohama-fa-marinos,kawasaki-frontale,5,21,1,3,2,...,-1.200000,1.000000,-0.800000,-0.400000,0.200000,-2.600000,-3.200000,16.800000,-3.280000,-3.06000
15010529,2015,1,2015-07-11,yokohama-fa-marinos,montedio-yamagata,5,29,1,1,0,...,-5.200000,-0.800000,0.600000,-0.600000,0.000000,-10.200000,-5.400000,-0.600000,-2.120000,6.18000
15010701,2015,1,2015-03-08,shimizu-s-pulse,kashima-antlers,7,1,3,1,1,...,0.200000,1.400000,-1.200000,-0.600000,0.200000,1.000000,-2.200000,-8.800000,-1.600000,0.46000
15010718,2015,1,2015-07-11,shimizu-s-pulse,vissel-kobe,7,18,0,5,2,...,2.800000,0.400000,-0.200000,-0.600000,0.000000,1.000000,1.800000,8.600000,0.300000,-0.42000
15010846,2015,1,2015-03-07,nagoya-grampus-eight,matsumoto-yamaga,8,46,3,3,0,...,8.804517,0.791261,-0.596109,0.400722,0.138731,0.662504,-0.770982,-6.622707,0.028553,-2.08254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20341121,2020,34,2020-12-19,kashiwa-reysol,kawasaki-frontale,11,21,2,3,2,...,9.000000,0.400000,-0.200000,1.000000,-0.200000,-24.200000,-12.200000,8.600000,-6.960000,-7.24000
20342218,2020,34,2020-12-19,fc-tokyo,vissel-kobe,22,18,1,0,1,...,7.400000,-1.800000,-0.800000,0.000000,0.000000,-26.600000,-1.400000,8.600000,-4.420000,-16.82000
20343331,2020,34,2020-12-19,sagan-tosu,oita-trinita,33,31,2,2,0,...,4.000000,-0.400000,1.200000,-1.000000,0.000000,14.800000,5.400000,1.600000,2.800000,-2.00000
20343405,2020,34,2020-12-19,yokohama-fc,yokohama-fa-marinos,34,5,3,1,1,...,-5.200000,-0.600000,0.000000,-1.600000,-0.200000,-10.000000,-8.200000,-17.600000,0.920000,1.76000


In [48]:
rf_preds = np.array(preds)
rf_preds

array([2, 1, 1, ..., 1, 2, 1])

In [49]:
df_analysis["RF"] = rf_preds

<ipython-input-49-db79f165e6c2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analysis["RF"] = rf_preds


In [50]:
df_analysis

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,...,InterceptsDiff,OffsidesDiff,YellowCardsDiff,RedCardsDiff,30mLineEntriesDiff,PenaltyAreaEntriesDiff,AttacksDiff,ChanceBuildingRateDiff,PossessionDiff,RF
15010521,2015,1,2015-03-07,yokohama-fa-marinos,kawasaki-frontale,5,21,1,3,2,...,1.000000,-0.800000,-0.400000,0.200000,-2.600000,-3.200000,16.800000,-3.280000,-3.06000,2
15010529,2015,1,2015-07-11,yokohama-fa-marinos,montedio-yamagata,5,29,1,1,0,...,-0.800000,0.600000,-0.600000,0.000000,-10.200000,-5.400000,-0.600000,-2.120000,6.18000,1
15010701,2015,1,2015-03-08,shimizu-s-pulse,kashima-antlers,7,1,3,1,1,...,1.400000,-1.200000,-0.600000,0.200000,1.000000,-2.200000,-8.800000,-1.600000,0.46000,1
15010718,2015,1,2015-07-11,shimizu-s-pulse,vissel-kobe,7,18,0,5,2,...,0.400000,-0.200000,-0.600000,0.000000,1.000000,1.800000,8.600000,0.300000,-0.42000,1
15010846,2015,1,2015-03-07,nagoya-grampus-eight,matsumoto-yamaga,8,46,3,3,0,...,0.791261,-0.596109,0.400722,0.138731,0.662504,-0.770982,-6.622707,0.028553,-2.08254,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20341121,2020,34,2020-12-19,kashiwa-reysol,kawasaki-frontale,11,21,2,3,2,...,0.400000,-0.200000,1.000000,-0.200000,-24.200000,-12.200000,8.600000,-6.960000,-7.24000,2
20342218,2020,34,2020-12-19,fc-tokyo,vissel-kobe,22,18,1,0,1,...,-1.800000,-0.800000,0.000000,0.000000,-26.600000,-1.400000,8.600000,-4.420000,-16.82000,2
20343331,2020,34,2020-12-19,sagan-tosu,oita-trinita,33,31,2,2,0,...,-0.400000,1.200000,-1.000000,0.000000,14.800000,5.400000,1.600000,2.800000,-2.00000,1
20343405,2020,34,2020-12-19,yokohama-fc,yokohama-fa-marinos,34,5,3,1,1,...,-0.600000,0.000000,-1.600000,-0.200000,-10.000000,-8.200000,-17.600000,0.920000,1.76000,2


In [51]:
# import csv
# import math
# import datetime
# import numpy as np
# import pandas as pd
# from decimal import Decimal
# import matplotlib.pyplot as plt
from scipy.stats import poisson
# from sklearn.metrics import log_loss
from dateutil.relativedelta import relativedelta

df_allmatch = pd.read_csv("./match_data_yearly/all_years.csv", index_col=0)
df_allmatch["Date"] = pd.to_datetime(df_allmatch["Date"])

def predict_with_poisson(match_id, match_range = 17):

    current_day = df_allmatch.at[match_id,"Date"]
    home = df_allmatch.at[match_id,"Home"]
    away = df_allmatch.at[match_id,"Away"]
    
    df_recent_home = df_allmatch[
        (df_allmatch["Date"] < current_day) &
        ((current_day.year-1)<= df_allmatch["Year"]) &
        (df_allmatch["Home"] == home) 
    ].tail(match_range)

    if  (match_range- len(df_recent_home)) :
        for _ in range(match_range- len(df_recent_home)):
            df_recent_home = df_recent_home.append({'Date':(current_day-relativedelta(days=1)) , 'HomeGF': 0.99, 'AwayGF': 1.66}, ignore_index=True)

    df_recent_away = df_allmatch[
        (df_allmatch["Date"] < current_day) &
        ((current_day.year-1)<= df_allmatch["Year"]) &
        (df_allmatch["Away"] == away)
    ].tail(match_range)

    if  (match_range- len(df_recent_away)) :
        for _ in range(match_range- len(df_recent_away)):
            df_recent_away = df_recent_away.append({'Date':(current_day-relativedelta(days=1)) , 'HomeGF': 1.83, 'AwayGF': 0.94}, ignore_index=True)

    beginning_day = min(df_recent_home.iloc[0,2],df_recent_away.iloc[0,2])

    df_recent = df_allmatch[
        (beginning_day <= df_allmatch["Date"] ) &
        (df_allmatch["Date"] < current_day) 
    ]

    homegf_ave = df_recent_home["HomeGF"].mean()
    homega_ave = df_recent_home["AwayGF"].mean()
    awaygf_ave = df_recent_away["AwayGF"].mean()
    awayga_ave = df_recent_away["HomeGF"].mean()
    homegf_league_ave = df_recent["HomeGF"].mean()
    awaygf_league_ave = df_recent["AwayGF"].mean()
    
#     print(f"{home}のホームでの平均得点 : {homegf_ave}")
#     print(f"{home}のホームでの平均失点 : {homega_ave}")
#     print(f"{away}のアウェイでの平均得点 : {awaygf_ave}")
#     print(f"{away}のアウェイでの平均失点 : {awayga_ave}")
#     print(f"リーグ全体のホームでの平均得点 : {homegf_league_ave}")
#     print(f"リーグ全体のアウェイでの平均得点 : {awaygf_league_ave}")
    
    home_attack = homegf_ave / homegf_league_ave
    away_defense = awayga_ave / homegf_league_ave
    homegf_pred = home_attack * away_defense * homegf_league_ave
    
    away_attack = awaygf_ave / awaygf_league_ave
    home_defense = homega_ave / awaygf_league_ave
    awaygf_pred = away_attack * home_defense * awaygf_league_ave
    
#     print(f"HA:{home_attack}, AA:{away_attack}, HD:{home_defense}, AD:{away_defense}, ")

#     print(f"得点予測  {home} {homegf_pred:.2f} - {awaygf_pred:.2f} {away}")
    
    x =  np.arange(0, 10, 1)

    home_poisson = poisson.pmf(x, homegf_pred)
    away_poisson = poisson.pmf(x, awaygf_pred)


#     plt.bar(x, home_poisson, align="edge", width=-0.4,  color="#7399D1", label=f'Home : {home}')
#     plt.bar(x, away_poisson, align="edge", width= 0.4,  color="#FA870A", label=f'Away : {away}')

#     plt.xlabel('Goals for')
#     plt.ylabel('Probability')
#     plt.grid(color='b', linestyle=':', linewidth=0.3)
#     plt.title(f"{match_id} {home} - {away}", color='rebeccapurple')
#     plt.legend()
#     plt.show()

    df_poisson = pd.DataFrame([home_poisson,away_poisson],columns=x, index=[home,away])
    
    
    win_prob_sum = 0
    loss_prob_sum = 0
    draw_prob_sum = 0
    
    for gf in x:
        gf_prob = df_poisson.at[home, gf]
        for ga in x:
            ga_prob = df_poisson.at[away,ga]
            score_prob = gf_prob * ga_prob

            if gf>ga:
                win_prob_sum += score_prob
            elif gf<ga:
                loss_prob_sum += score_prob
            else:
                draw_prob_sum += score_prob

    prob_sum = win_prob_sum+loss_prob_sum+draw_prob_sum
    win_prob_sum = win_prob_sum/prob_sum
    loss_prob_sum = loss_prob_sum/prob_sum
    draw_prob_sum = draw_prob_sum/prob_sum

#     print(f"{home} - {away}")
#     print(f"win {win_prob_sum:.2%}")
#     print(f"loss {loss_prob_sum:.2%}")
#     print(f"draw {draw_prob_sum:.2%}")

    return draw_prob_sum,win_prob_sum,loss_prob_sum

In [52]:
predict_with_poisson(19071307)

(0.22582333767491594, 0.3617548608605765, 0.4124218014645075)

In [53]:
wl_tests=[]
wl_preds=[]
loglosses = []
accuracies = []
correct_preds = []
draw_correct_count = 0

for y in range(2015,2021):
    df_pred = pd.DataFrame(index=df_allmatch[df_allmatch["Year"] == y].index, columns=[0,1,2,"pred","actual"])
    correct_count = 0
    y_test = []
    y_pred_proba = []

    for index, row in df_pred.iterrows():   

        d_pred = predict_with_poisson(index)[0]
        w_pred = predict_with_poisson(index)[1]
        l_pred = predict_with_poisson(index)[2]

        if max(predict_with_poisson(index)) == w_pred:
            wl_pred = 1
        elif max(predict_with_poisson(index)) == l_pred:
            wl_pred = 2
        else:
            wl_pred = 0
            
        wl_preds.append(wl_pred)

        actual = df_allmatch.at[index, "W/L"]
        
        wl_tests.append(actual)

        row[0] = d_pred
        row[1] = w_pred
        row[2] = l_pred
        row["pred"] = wl_pred
        row["actual"] = actual
        
        y_pred_proba.append([d_pred,w_pred,l_pred])
        y_test.append(actual)
        
        if wl_pred == actual:
            correct_count += 1
            correct_preds.append(wl_pred)
    
    logloss = log_loss(y_test,y_pred_proba,labels=[0,1,2])
    loglosses.append(logloss)

    accuracy = correct_count/len(df_pred)
    accuracies.append(accuracy)
    print(f"{y} 的中率:{accuracy:.3f}, 対数損失:{logloss:.3f}")
    
print("==============================================")
print(f"的中率(Accuracy) : {np.mean(accuracies)}" )
print(f"対数損失(Log loss) : {np.mean(loglosses)}" )


2015 的中率:0.480, 対数損失:1.052
2016 的中率:0.474, 対数損失:1.056
2017 的中率:0.461, 対数損失:1.081
2018 的中率:0.431, 対数損失:1.129
2019 的中率:0.448, 対数損失:1.090
2020 的中率:0.477, 対数損失:1.067
的中率(Accuracy) : 0.46187363834422657
対数損失(Log loss) : 1.079116976898283


In [54]:
poisson_preds = wl_preds

In [55]:
df_analysis["Poisson"] = poisson_preds
df_analysis

<ipython-input-55-f777a1180515>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analysis["Poisson"] = poisson_preds


,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,...,OffsidesDiff,YellowCardsDiff,RedCardsDiff,30mLineEntriesDiff,PenaltyAreaEntriesDiff,AttacksDiff,ChanceBuildingRateDiff,PossessionDiff,RF,Poisson
15010521,2015,1,2015-03-07,yokohama-fa-marinos,kawasaki-frontale,5,21,1,3,2,...,-0.800000,-0.400000,0.200000,-2.600000,-3.200000,16.800000,-3.280000,-3.06000,2,1
15010529,2015,1,2015-07-11,yokohama-fa-marinos,montedio-yamagata,5,29,1,1,0,...,0.600000,-0.600000,0.000000,-10.200000,-5.400000,-0.600000,-2.120000,6.18000,1,1
15010701,2015,1,2015-03-08,shimizu-s-pulse,kashima-antlers,7,1,3,1,1,...,-1.200000,-0.600000,0.200000,1.000000,-2.200000,-8.800000,-1.600000,0.46000,1,1
15010718,2015,1,2015-07-11,shimizu-s-pulse,vissel-kobe,7,18,0,5,2,...,-0.200000,-0.600000,0.000000,1.000000,1.800000,8.600000,0.300000,-0.42000,1,1
15010846,2015,1,2015-03-07,nagoya-grampus-eight,matsumoto-yamaga,8,46,3,3,0,...,-0.596109,0.400722,0.138731,0.662504,-0.770982,-6.622707,0.028553,-2.08254,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20341121,2020,34,2020-12-19,kashiwa-reysol,kawasaki-frontale,11,21,2,3,2,...,-0.200000,1.000000,-0.200000,-24.200000,-12.200000,8.600000,-6.960000,-7.24000,2,2
20342218,2020,34,2020-12-19,fc-tokyo,vissel-kobe,22,18,1,0,1,...,-0.800000,0.000000,0.000000,-26.600000,-1.400000,8.600000,-4.420000,-16.82000,2,1
20343331,2020,34,2020-12-19,sagan-tosu,oita-trinita,33,31,2,2,0,...,1.200000,-1.000000,0.000000,14.800000,5.400000,1.600000,2.800000,-2.00000,1,2
20343405,2020,34,2020-12-19,yokohama-fc,yokohama-fa-marinos,34,5,3,1,1,...,0.000000,-1.600000,-0.200000,-10.000000,-8.200000,-17.600000,0.920000,1.76000,2,2


In [56]:
df_analysis[df_analysis["RF"] == df_analysis["Poisson"]]

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,...,OffsidesDiff,YellowCardsDiff,RedCardsDiff,30mLineEntriesDiff,PenaltyAreaEntriesDiff,AttacksDiff,ChanceBuildingRateDiff,PossessionDiff,RF,Poisson
15010529,2015,1,2015-07-11,yokohama-fa-marinos,montedio-yamagata,5,29,1,1,0,...,0.600000,-0.600000,0.000000,-10.200000,-5.400000,-0.600000,-2.120000,6.18000,1,1
15010701,2015,1,2015-03-08,shimizu-s-pulse,kashima-antlers,7,1,3,1,1,...,-1.200000,-0.600000,0.200000,1.000000,-2.200000,-8.800000,-1.600000,0.46000,1,1
15010718,2015,1,2015-07-11,shimizu-s-pulse,vissel-kobe,7,18,0,5,2,...,-0.200000,-0.600000,0.000000,1.000000,1.800000,8.600000,0.300000,-0.42000,1,1
15010846,2015,1,2015-03-07,nagoya-grampus-eight,matsumoto-yamaga,8,46,3,3,0,...,-0.596109,0.400722,0.138731,0.662504,-0.770982,-6.622707,0.028553,-2.08254,1,1
15010928,2015,1,2015-07-11,gamba-osaka,ventforet-kofu,9,28,2,1,1,...,-0.400000,0.400000,0.000000,12.400000,4.800000,5.000000,3.280000,5.56000,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20340314,2020,34,2020-12-19,urawa-red-diamonds,consadole-sapporo,3,14,0,2,2,...,-0.600000,0.200000,0.000000,-15.400000,-2.000000,-5.800000,-2.240000,-4.94000,2,2
20340810,2020,34,2020-12-19,nagoya-grampus-eight,sanfrecce-hiroshima,8,10,1,0,1,...,-2.400000,-0.400000,0.000000,-8.600000,-2.000000,-9.400000,-0.760000,-1.42000,1,1
20341121,2020,34,2020-12-19,kashiwa-reysol,kawasaki-frontale,11,21,2,3,2,...,-0.200000,1.000000,-0.200000,-24.200000,-12.200000,8.600000,-6.960000,-7.24000,2,2
20343405,2020,34,2020-12-19,yokohama-fc,yokohama-fa-marinos,34,5,3,1,1,...,0.000000,-1.600000,-0.200000,-10.000000,-8.200000,-17.600000,0.920000,1.76000,2,2


In [57]:
df_analysis[df_analysis["RF"] == df_analysis["Poisson"]]["RF"].value_counts()

1    582
2    349
Name: RF, dtype: int64

In [58]:
len(df_analysis[df_analysis["RF"] == df_analysis["Poisson"]])/len(df_analysis)

0.5070806100217865

In [59]:
df_analysis[(df_analysis["RF"] == df_analysis["Poisson"]) & (df_analysis["RF"] == df_analysis["W/L"])]

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,...,OffsidesDiff,YellowCardsDiff,RedCardsDiff,30mLineEntriesDiff,PenaltyAreaEntriesDiff,AttacksDiff,ChanceBuildingRateDiff,PossessionDiff,RF,Poisson
15010701,2015,1,2015-03-08,shimizu-s-pulse,kashima-antlers,7,1,3,1,1,...,-1.2,-0.6,0.2,1.0,-2.2,-8.8,-1.60,0.46,1,1
15010928,2015,1,2015-07-11,gamba-osaka,ventforet-kofu,9,28,2,1,1,...,-0.4,0.4,0.0,12.4,4.8,5.0,3.28,5.56,1,1
15011208,2015,1,2015-07-11,shonan-bellmare,nagoya-grampus-eight,12,8,2,1,1,...,0.4,-0.2,-0.2,7.6,4.0,-0.2,6.16,-0.70,1,1
15017801,2015,1,2015-07-11,albirex-niigata,kashima-antlers,78,1,2,3,2,...,0.0,0.0,0.0,-11.2,-2.8,5.0,-4.90,-4.90,2,2
15020329,2015,2,2015-03-14,urawa-red-diamonds,montedio-yamagata,3,29,1,0,1,...,-2.0,-0.2,0.0,-3.8,6.2,-10.2,0.78,3.82,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20332214,2020,33,2020-11-11,fc-tokyo,consadole-sapporo,22,14,1,0,1,...,-1.6,0.8,0.2,-7.4,-0.6,-0.8,2.62,-9.20,1,1
20333409,2020,33,2020-12-16,yokohama-fc,gamba-osaka,34,9,0,2,2,...,0.0,-0.2,0.0,0.0,-4.0,-11.0,0.52,4.26,2,2
20340314,2020,34,2020-12-19,urawa-red-diamonds,consadole-sapporo,3,14,0,2,2,...,-0.6,0.2,0.0,-15.4,-2.0,-5.8,-2.24,-4.94,2,2
20340810,2020,34,2020-12-19,nagoya-grampus-eight,sanfrecce-hiroshima,8,10,1,0,1,...,-2.4,-0.4,0.0,-8.6,-2.0,-9.4,-0.76,-1.42,1,1


In [60]:
df_analysis[(df_analysis["RF"] == df_analysis["Poisson"]) & (df_analysis["RF"] == df_analysis["W/L"])]["W/L"].value_counts()

1    278
2    169
Name: W/L, dtype: int64

In [61]:
df_analysis[(df_analysis["RF"] == df_analysis["Poisson"]) & (df_analysis["RF"] == df_analysis["W/L"])]

,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,...,OffsidesDiff,YellowCardsDiff,RedCardsDiff,30mLineEntriesDiff,PenaltyAreaEntriesDiff,AttacksDiff,ChanceBuildingRateDiff,PossessionDiff,RF,Poisson
15010701,2015,1,2015-03-08,shimizu-s-pulse,kashima-antlers,7,1,3,1,1,...,-1.2,-0.6,0.2,1.0,-2.2,-8.8,-1.60,0.46,1,1
15010928,2015,1,2015-07-11,gamba-osaka,ventforet-kofu,9,28,2,1,1,...,-0.4,0.4,0.0,12.4,4.8,5.0,3.28,5.56,1,1
15011208,2015,1,2015-07-11,shonan-bellmare,nagoya-grampus-eight,12,8,2,1,1,...,0.4,-0.2,-0.2,7.6,4.0,-0.2,6.16,-0.70,1,1
15017801,2015,1,2015-07-11,albirex-niigata,kashima-antlers,78,1,2,3,2,...,0.0,0.0,0.0,-11.2,-2.8,5.0,-4.90,-4.90,2,2
15020329,2015,2,2015-03-14,urawa-red-diamonds,montedio-yamagata,3,29,1,0,1,...,-2.0,-0.2,0.0,-3.8,6.2,-10.2,0.78,3.82,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20332214,2020,33,2020-11-11,fc-tokyo,consadole-sapporo,22,14,1,0,1,...,-1.6,0.8,0.2,-7.4,-0.6,-0.8,2.62,-9.20,1,1
20333409,2020,33,2020-12-16,yokohama-fc,gamba-osaka,34,9,0,2,2,...,0.0,-0.2,0.0,0.0,-4.0,-11.0,0.52,4.26,2,2
20340314,2020,34,2020-12-19,urawa-red-diamonds,consadole-sapporo,3,14,0,2,2,...,-0.6,0.2,0.0,-15.4,-2.0,-5.8,-2.24,-4.94,2,2
20340810,2020,34,2020-12-19,nagoya-grampus-eight,sanfrecce-hiroshima,8,10,1,0,1,...,-2.4,-0.4,0.0,-8.6,-2.0,-9.4,-0.76,-1.42,1,1
